In [1]:
# imports
from urllib.request import urlopen
import json
import re
from urllib.parse import quote
import pandas as pd
from pandas.io.json import json_normalize
import collections
import itertools
import networkx as nx
import matplotlib.pyplot as plt
from fa2 import ForceAtlas2
import numpy as np
import datetime
from lxml import html
import glob
import ast # string to dirc
import math # check if float is NaN
# conda install -c phlya adjusttext 
#from adjustText import adjust_text

In [ ]:
###### Save inital data in top-wikis-date #####
n = 250

# Initial query - get top wikis by pageviews using API
query = "http://www.wikia.com/api/v1/Wikis/List?expand=1&limit="+str(n)+"&batch=1"
response = urlopen(query)
wikisource = response.read()
data = json.loads(wikisource)

# Save data at data/wikis/top-wikis-date.json
date = datetime.date.today().strftime("%d-%m-%Y")
with open("data/wikis/top-wikis-"+date+".json", 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
##### Read initial wiki data  ######
date= 
df = pd.DataFrame()
with open("data/wikis/top-wikis-"+date+".json") as f:
    json_file = json.loads(f.read())
df = pd.DataFrame(json_file['items']) 

In [ ]:
# Search for more wikis (primarily based on the different hub names)
def saveSearchWikis(search, n):
    querySearch = "http://www.wikia.com/api/v1/Wikis/ByString?expand=1&string="+str(search)+"&limit="+str(n)+"&batch=1&includeDomain=true"
    response = urlopen(querySearch)
    data = json.loads(response.read())
    
    if data['total'] > 0:
        date = datetime.date.today().strftime("%d-%m-%Y")
        searchWikiPath = "data/wikis/search-"+search+"-wikis-"+date+".json"
        with open(searchWikiPath, 'w') as outfile:
            json.dump(data, outfile)
    else:
        print("Data not found in %s: %s" % (search, data['items']))
    return pd.DataFrame(data['items']) 

In [ ]:
##### Use hubs in dataset to searsh for new wikis and save as search-hub-wikis ##### 
fullDataSet = pd.DataFrame()
for hub in set(df["hub"]):
    df = saveSearchWikis(hub, n)
    fullDataSet = fullDataSet.append(df)
    print("%s wikis found in %s" % (str(len(df)), hub))
fullDataSet = fullDataSet.reset_index(drop=True)

In [2]:
# search online to find the username based on the userID
def saveUsernames(keyword):
    keyword = str(keyword)
    querySearch = "http://community.wikia.com/wiki/Special:Search?search="+keyword+"&fulltext=Search&ns2=1"
    response = urlopen(querySearch)
    wikisource = response.read()
    usernames = re.findall("wiki\/User:([(\w)]+)?",str(wikisource))
    
    # Read exsisting data to avoid duplicates
    filename = "data/users/userNames.txt"
    fileRead = open(filename, "r")
    contents = fileRead.read()
    fileRead.close() 
    
    # Print new usernames
    for user in set(usernames):
        userData = user+"\n"
        if userData not in contents: # not a duplicate
            fileWrite = open(filename,"a")
            fileWrite.write(userData)
            fileWrite.close()

    return list(set(usernames))

In [27]:
def userIDtoUserName(userID):
    if userID == None:
        print("user ID is %s" % userID)
        return []
    
    if int(userID) < 1:
        print("user ID less than 1: %s" % userID)
        return []
    
    userID = str(userID)
    querySearch = "http://www.wikia.com/api/v1/User/Details?ids="+userID
    response = urlopen(querySearch)
    wikisource = response.read()
    data = json.loads(wikisource)
    
    usernames = []
    # Print new usernames
    for name in data['items']:
        usernames.append(name['title'])
    return usernames


In [ ]:
# search online to find the username based on the userID
def saveUsernamesByUserID(userID):
    
    if userID == None:
        print("user ID is %s" % userID)
        return []
    
    if int(userID) < 1:
        print("user ID less than 1: %s" % userID)
        return []
    
    userID = str(userID)
    querySearch = "http://www.wikia.com/api/v1/User/Details?ids="+userID
    response = urlopen(querySearch)
    wikisource = response.read()
    data = json.loads(wikisource)
    
    # Read exsisting data to avoid duplicates
    filename = "data/users/userNames.txt"
    fileRead = open(filename, "r")
    contents = fileRead.read()
    fileRead.close() 
    
    # Print new usernames
    for name in data['items']:
        userData = name['title']+" "+name['user_id']+"\n"
        if userData not in contents: # not a duplicate
            fileWrite = open(filename,"a")
            fileWrite.write(userData)
            fileWrite.close()
        elif name['title'] in contents:
            content.replace(name['title'],userData)
            fileWrite = open(filename,"w")
            fileWrite.write(content)
            fileWrite.close()


In [ ]:
# search online to find the wiki names which the user use, 
# This wiki saves the users wikis and all the wiki names
def saveNewWikisAndUsersThroughUser(usernames):
    wikis = []
    for username in usernames:
        
        # encode username correct
        username = str(username.encode('utf-8'))
        username = username.replace("'", "")
        username = username.replace("b", "")
        
        # find users activity
        querySearch = "http://community.wikia.com/index.php?limit=1000&tagfilter=&title=Special%3AContributions&target="+username+"&namespace=&tagfilter=&year=&month=-1.html"
        response = urlopen(querySearch)
        wikisource = response.read()
        
        
        # search html for wikis
        usersWikis = re.findall("wiki/Adoption:(\w+)?",str(wikisource))
        usersWikis = [wiki for wiki in usersWikis if (wiki != "Requests" and wiki != '')]
        usersWikis = list(set(usersWikis))
        
        # Save users and the individual user's wikis 
        saveUsersWikis(username, usersWikis)
        
        [wikis.append(wiki) for wiki in usersWikis]
        wikis = list(set(wikis))
        
    # Save all found wiki names and read exsisting data to avoid duplicates
    filename = "data/wikis/wikiNames.txt"
    fileRead = open(filename, "r")
    contents = fileRead.read()
    fileRead.close() 
    for wiki in set(wikis):
        wikiData = wiki+"\n"
        if wikiData not in contents:
            # Print new usernames
            fileWrite = open(filename,"a")
            fileWrite.write(wikiData)
            fileWrite.close()
    
    return wikis

In [ ]:
def collectAndSaveData(userID):
    # Get and save all usernames
    saveUsernamesByUserID(userID)
    
    # Get and save all wikis found connected to the usernames
    wikiList = saveNewWikisAndUsersThroughUser(username)


In [4]:
# remove inactive wikis and duplicates
def findActiveWikis(dataSet):
    activeDataSet = pd.DataFrame()
    dataSet = dataSet.drop_duplicates(subset="id")
    # find stats in dataset
    for s in dataSet['stats']:
        # only add when at least 1 active
        print(s['activeUsers'])
        if int(s['activeUsers'])>1:
            data = dataSet.loc[dataSet['stats']==s]

            activeDataSet = activeDataSet.append(data)#    
    # remove duplicates
    #activeDataSet = activeDataSet.reset_index(drop=True)
    return activeDataSet

In [ ]:
# Get active wikis
activeDataSet = findActiveWikis(fullDataSet)
print("full data set has %s data, while only %s is actively used." % (len(fullDataSet),len(activeDataSet)))
print()


# Find more users and their connected Wikis
# find topusers
topUsers = [x for x in activeDataSet["topUsers"]]
topUsers = itertools.chain.from_iterable(topUsers)

# find founders
founders = [x for x in activeDataSet['founding_user_id']]

# all the different userID
users = list(set(topUsers + founders))

# find the wiki names of the users
userWiki = {}
for user in users:
    for k, v in fullDataSet.T.items():
        if user == v["topUsers"] or user == v['founding_user_id']:
            if user not in userWiki:
                userWiki[user] = []
            userWiki[user].append(v['name'])

userWiki

In [ ]:
# find additional users connections

for i in range(0,len(users)):
    collectAndSaveData(users[i])
    if i % 500 == 0:
        print(i, len(users))

In [ ]:
##### add Wiki Names ######
# 12.21
# Read exsisting data to avoid duplicates
wikiNames = []
filename = "data/wikis/wikiNames.txt"
f = open(filename, "r")
for line in f.readlines():
    wikiNames.append(line.replace("\n",""))
f.close() 

i = 0
for wikiName in wikiNames:
    i+=1
    df = saveSearchWikis(wikiName, n)
    fullDataSet = fullDataSet.append(df)
    if i%100==0:
        print("%i af %s" % (i,len(wikiNames)))

fullDataSet = fullDataSet.drop_duplicates(subset="name")    
fullDataSet = fullDataSet.reset_index(drop=True)    
len(fullDataSet)

In [ ]:
##### Read Wiki Names ######
# 22:49 - 
wikiDataPath = [f for f in glob.glob("data/wikis/*.json")]
print(len(wikiDataPath))
i = 0
for dataPath in wikiDataPath:
    i+=1
    with open(dataPath) as f:
        json_file = json.loads(f.read())
    fullDataSet = fullDataSet.append(pd.DataFrame(json_file['items']))
    if(i%200 ==0):
        print(i)

fullDataSet = fullDataSet.drop_duplicates(subset="id")    
fullDataSet = fullDataSet.reset_index(drop=True) 

print(len(fullDataSet))
fullDataSet = findActiveWikis(fullDataSet)
print(len(fullDataSet))

In [ ]:
# Save Collected data
date = datetime.date.today().strftime("%d-%m-%Y")
fullDataSet.to_csv("data/sortedWikiData-"+date+".csv", sep='\t')

In [5]:
# remove inactive wikis and duplicates
def findActiveWikis(dataSet):
    activeDataSet = pd.DataFrame()
    dataSet = dataSet.drop_duplicates(subset="id")
    # find stats in dataset
    for s in dataSet['stats']:
        # only add when at least 1 active
        sDirc = ast.literal_eval(s)
        if int(sDirc['activeUsers'])>1:
            data = dataSet.loc[dataSet['stats']==s]
            activeDataSet = activeDataSet.append(data)#    
    # remove duplicates
    #activeDataSet = activeDataSet.reset_index(drop=True)
    return activeDataSet

In [6]:
### Get Data from CSV ###

fullDataSet = pd.read_csv("data/sortedWikiData.csv", sep='\t')

5849

In [ ]:
### Filter Data ###
activeDataSet = findActiveWikis(fullDataSet)
len(activeDataSet)

In [11]:
# Find more users and their connected Wikis
# find topusers
topUsers = []
for users in activeDataSet["topUsers"]:
    userList = makeStringToList(users)
    for user in userList: 
        topUsers.append(str(int(user)))

# find founders
founders = []
for user in activeDataSet["founding_user_id"]:
    user = floatToString(user)
    if int(user) > -1:
        founders.append(user)

# all the different userID
users = list(set(topUsers + founders))
len(users)

22472

In [15]:
# find the wiki names of the users
userWiki = {}
i = 0
for user in users:
    i+= 1
    for k, v in activeDataSet.T.items():
        topUsers = makeStringToList(v["topUsers"])
        if user in topUsers or user == floatToString(v['founding_user_id']):
            if user not in userWiki:
                if i % 500 == 0:
                    print(i, end=", ")
                userWiki[user] = []
            userWiki[user].append(v['name'])
            
len(userWiki)

500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000, 10500, 11000, 11500, 12000, 12500, 13000, 13500, 14000, 14500, 15000, 15500, 16000, 16500, 17000, 17500, 18000, 18500, 19000, 19500, 20000, 20500, 21000, 21500, 22000, 

22472

In [31]:
for k, v in userWiki.items():
    name = userIDtoUserName(k)
    if len(name) > 0:
        saveUsersWikis(name[0], v)
    else:
        print(name)

user ID less than 1: 0
[]


In [22]:
# saving new users in file user.wikis
def saveUsersWikis (username, userWikis):
    userData = username+": "+str(userWikis)+"\n"
        
    filename = "data/users/user-wikis.txt"
    fileRead = open(filename, "r")
    contents =fileRead.read()
        
    # Ensure no duplicates
    if userData not in contents: # not a duplicate
        fileWrite = open(filename,"a")
        fileWrite.write(userData)
        fileWrite.close()
        fileRead.close() 

In [9]:
def makeStringToList(string):
    if string == "[]":
        return []
    l = string.replace("[","")
    l = l.replace("]","")
    l = l.split(", ")
    return l

len(users)
list(set(users[user[0]]))

In [10]:
def floatToString(f):
    if not math.isnan(f):
        return str(int(f))
    else:
        return "-1"

In [ ]:
# find the wiki names of the users
usersWikis = {}
print(len(activeDataSet.T.items()))
for k, v in activeDataSet.T.items():
    topUsers = makeStringToList(v["topUsers"])
    founder = floatToString(v['founding_user_id']
    userWikiData = list(set(topUsers.append(founder)))
    for user in userWikiData:                       
        if user not in usersWikis and user != "-1":
            if i % 500 == 0:
                print(i, end=", ")
                usersWikis[user] = []
            usersWikis[user].append(v['name'])
            
len(usersWikis)